In [1]:
import pandas as pd
from xml.etree import ElementTree
from xml.etree.ElementTree import Element

import os
import sqlalchemy
import mercante


MERCANTE = 'mercante'

2019-10-07 11:14:23,815 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  /home/ivan/pybr/ajna_api/ajna_commons/flask/error.log
Fazendo log de atividade no arquivo  /home/ivan/pybr/ajna_api/ajna_commons/flask/access.log


In [2]:
lista_arquivos = os.listdir(MERCANTE)

In [4]:
print(lista_arquivos[0])

MERCANTE_CONHECIMENTO_2019-09-27-16-00-00.xml


In [229]:
with open(os.path.join(MERCANTE, lista_arquivos[1]), 'r', encoding='iso-8859-1') as file1:
    content1 = file1.read()

In [230]:
xtree = ElementTree.parse(os.path.join(MERCANTE, lista_arquivos[1]))
xroot = xtree.getroot()

In [ ]:
print(xroot)
for node in xroot:
    print(node)
    print(node.tag)
    if isinstance(node, Element):
        for subnode in node:
            print(subnode.tag)
            print(subnode.text)


In [216]:
class ParseXML:

    def _campos(self):
        return [campo for campo in dir(self) if campo[0] != '_']

    def _parse_node(self, node):
        for campo in self._campos():
            # print(campo)
            alvo = node.find(campo) 
            if alvo is not None:
                # print(alvo.text, alvo.tag)
                destino = getattr(self, campo)
                if isinstance (destino, str):
                    setattr(self, campo, alvo.text)
                else:
                    setattr(self, campo, alvo)
            else:
                print('Não encontrou %s' % campo)

    def _to_dict(self):
        result = {}
        for campo in self._campos():
            valor = getattr(self, campo)
            if isinstance(valor, ParseXML):
                valor = str(valor)
            result[campo] = valor 
        return result
        

class Embarcador(ParseXML):
    cnpjShipper = ''
    idEmbarcador = ''

    def __repr__(self):
        return self.cnpjShipper
    
class Consignatario(ParseXML):
    tipoConsignatario = ''
    cnpjConsignatario = ''
    dadosComplementaresConsignatario = ''

    def __repr__(self):
        return self.cnpjConsignatario

class Conhecimento(ParseXML):
    def __init__(self):
        self.tipoMovimento:str = ''
        self.dataAtualizacao:str = ''
        self.horaAtualizacao:str = ''
        self.tipoTrafego:str = ''
        self.tipoBLConhecimento:str = ''
        self.numeroCEMaster:str = ''
        self.dataEmissao:str = ''
        self.cubagem:str = ''
        self.portoDestFinal:str = ''
        self.portoOrigemCarga:str = ''
        self.descricao:str = ''
        self.numeroCEmercante:str = ''
        self._embarcador:Element = None
        self._consignatario:Element = None

    
    @property
    def embarcador(self):
        return self._embarcador
        
    @embarcador.setter
    def embarcador(self, node: Element):
        self._embarcador = Embarcador()
        self._embarcador._parse_node(node)

    @property
    def consignatario(self):
        return self._consignatario
        
    @consignatario.setter
    def consignatario(self, node: Element):
        self._consignatario = Consignatario()
        self._consignatario._parse_node(node)

        
    # def __repr__(self):
    #    repr = ' '.join([ campo + '-' + getattr(self, campo) for campo in self._campos()])
    #    if repr is None:
    #        return 'ERRO'
    #    return repr
        

manifestosCarga
<tipoMovimento>A</tipoMovimento>
<numero>1919301835474</numero>
<codAgenciaInformante>02427026000146</codAgenciaInformante>
<codigoEmpresaNavegacao>02427026000146</codigoEmpresaNavegacao>
<numeroImoDPC>9625396</numeroImoDPC>
<dataEncerramento>2019-09-13</dataEncerramento>
<dataInicioOperacao>2019-09-27</dataInicioOperacao>
<portoCarregamento>BRSSA</portoCarregamento>
<portoDescarregamento>BRIBB</portoDescarregamento>
<tipoTrafego>3</tipoTrafego>
<numeroViagem>937S</numeroViagem>
<quantidadeConhecimento>7</quantidadeConhecimento>
<dataAtualizacao>2019-09-27</dataAtualizacao>
<horaAtualizacao>00:01:18</horaAtualizacao>
<codigoTerminalCarregamento>BRSSA002</codigoTerminalCarregamento>
<codigoTerminalDescarregamento>BRIBB002</codigoTerminalDescarregamento>
<numeroEscalaMercante>19000279154</numeroEscalaMercante><numeroEscalaMercante>19000279162</numeroEscalaMercante><numeroEscalaMercante>19000279170</numeroEscalaMercante><numeroEscalaMercante>19000279189</numeroEscalaMercante><numeroEscalaMercante>19000336905</numeroEscalaMercante>

In [217]:
Conhecimento()._campos()

['consignatario',
 'cubagem',
 'dataAtualizacao',
 'dataEmissao',
 'descricao',
 'embarcador',
 'horaAtualizacao',
 'numeroCEMaster',
 'numeroCEmercante',
 'portoDestFinal',
 'portoOrigemCarga',
 'tipoBLConhecimento',
 'tipoMovimento',
 'tipoTrafego']

In [232]:
classes = {'conhecimentosEmbarque': Conhecimento}
    
    
objetos = []
for node in xroot:
    classe = classes.get(node.tag)
    if classe:
        # print(node.tag, node.text)
        objeto = classe()
        objeto._parse_node(node)
        # objeto = Conhecimento()
        # objeto._parse_node(node)
        # print(objeto)
        objetos.append(objeto._to_dict())
        
        # break

In [233]:
[(campo, getattr(objeto, campo)) for campo in objeto._campos()]

[('consignatario', None),
 ('cubagem', '0000000135.000'),
 ('dataAtualizacao', '2019-09-27'),
 ('dataEmissao', '2019-09-23'),
 ('descricao',
  '1 CONTAINER WITH                        ARGENTINE KABULI CHICK PEAS             GRADE 2 CROP 2019                       1 CONTAINER WITH                        ARGENTINE KABULI CHICK PEAS             GRADE 2 CROP 2019                       1 CONTAINER WITH                        ARGENTINE KABULI CHICK PEAS             GRADE 2 CROP 2019                       1 CONTAINER WITH                        ARGENTINE KABULI CHICK PEAS             GRADE 2 CROP 2019                       1 CONTAINER WITH'),
 ('embarcador', ),
 ('horaAtualizacao', '15:59:21'),
 ('numeroCEMaster', ''),
 ('numeroCEmercante', '001909034537706'),
 ('portoDestFinal', 'INNIP'),
 ('portoOrigemCarga', 'ARZAE'),
 ('tipoBLConhecimento', '10'),
 ('tipoMovimento', 'A'),
 ('tipoTrafego', '9')]

In [234]:
objeto.embarcador

In [235]:
objeto.embarcador.cnpjShipper

''

In [236]:
df_conhecimentos = pd.DataFrame(objetos)

In [237]:
df_conhecimentos.head()

""


In [238]:
df_conhecimentos.shape

(0, 0)

In [239]:
len(df_conhecimentos.numeroCEmercante.value_counts())

AttributeError: 'DataFrame' object has no attribute 'numeroCEmercante'

In [226]:
df_conhecimentos.tipoMovimento.value_counts()

I    397
A    107
E      3
Name: tipoMovimento, dtype: int64

In [227]:
!pip install mysql-connector-python

In [18]:
arquivo = lista_arquivos[1]
print(arquivo)
xtree = ElementTree.parse(os.path.join(mercante.MERCANTE_DIR, arquivo))
xroot = xtree.getroot()
objetos = []
for node in xroot:
    classe = mercante.classes_em_lista.get(node.tag)
    if classe:
        last_classname = classe
        classe_pai = mercante.classes.get(node.tag)
        objeto_pai = classe_pai()
        objeto_pai._parse_node(node)
        tag_classe = classe.tag
        for subnode in node.findall(tag_classe):
            objeto = classe(objeto_pai)
            objeto._parse_node(subnode)
            objetos.append(objeto._to_dict())
print(objetos)

MERCANTE_ITEM_2019-09-27-15-20-00.xml
[{'codigoConteiner': 'MSCU4459376', 'codigoTipoEmbalagem': '01', 'descritivo': 'MSCUPA427140', 'identificacaoNCM': '8901', 'itemEmbaladoMadeira': 'false', 'marcaMercadoria': 'MSCU4459376', 'numeroCEMercante': '151905216436912', 'numeroIdentificacao': '00001', 'numeroSequencialItemCarga': '0001', 'qtdeVolumes': '000001', 'tag': 'NCM'}, {'codigoConteiner': 'MSCU4466591', 'codigoTipoEmbalagem': '01', 'descritivo': 'MSCUPA427140', 'identificacaoNCM': '8901', 'itemEmbaladoMadeira': 'false', 'marcaMercadoria': 'MSCU4466591', 'numeroCEMercante': '151905216436912', 'numeroIdentificacao': '00001', 'numeroSequencialItemCarga': '0002', 'qtdeVolumes': '000001', 'tag': 'NCM'}, {'codigoConteiner': 'TGBU7058381', 'codigoTipoEmbalagem': '', 'descritivo': '', 'identificacaoNCM': '1509', 'itemEmbaladoMadeira': '', 'marcaMercadoria': '', 'numeroCEMercante': '181905216443406', 'numeroIdentificacao': '', 'numeroSequencialItemCarga': '0001', 'qtdeVolumes': '', 'tag': 'N

In [19]:
import sqlalchemy

engine = sqlalchemy.create_engine(
    'mysql+mysqlconnector://ivan@localhost:3306/mercante')

In [20]:
print(objeto)
print(objeto.__class__.__name__)


27101921
NCMItemCarga


In [21]:
df = pd.DataFrame(objetos)

df.to_sql(objeto.__class__.__name__, engine, if_exists='replace')